#PRACTICAL SESSION 2

**PART I LOGISTIC REGRESSION**

STEP 1: Importing the libraries

In [ ]:
# Important libraries
import csv
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import minimize
import math
from data_analysis import Data,buildmodel,PlotBoundary,testAccuracy

STEP 2: Defining a class to create usable data

In [ ]:
# Lets create the object data and plot the correlation matrix
data = Data('P2_microchip_logreg.csv')
data.splitData(0.2)
data.plotCorrelationMatrix()

STEP 3: Solving the logistic regression

In [ ]:
# Define the hypothesis function between X and theta
def hypothesisFunction(X,theta):
  
  return h

# Define a function that applies sigmoid element-wise
def sigmoid(z):

  return g

# Write a function that returns a scalar with inputs: predictions "P" and targets "T", using negative loglikelihood 
def crossentroy(P,T):
  
  return L

# Define the function that given the parameter lambda and theta returns the regularization term
def regularization(l,theta):
  
  return r

In [ ]:
# Gather all the functions above to create a function that given a set of parameters (theta, the data X, the labels Y and the value of 
# lambda), it returns the cost of the negative loglikelihood and another function which returns its gradient.
def costFunction(theta,X,Y,l):
  
  return J
  
def gradient(theta,X,Y,l):
  
  return grad

In [ ]:
# We use build model to create the desired matrices both for train and test. Then minimize function from scipy is used to 
# to obtain the optimum parameters
# Try different values of data augmentation to observe the boundaries
d = 1
l = 0
theta0,Xtr,Ytr = buildmodel(data.Xtrain,data.Ytrain,d,[0,1])
_,Xte,Yte = buildmodel(data.Xtest,data.Ytest,d,[0,1])
sol = minimize(costFunction,theta0,args=(Xtr,Ytr,l))
theta = np.reshape(sol.x,(Xtr.shape[1],Ytr.shape[1]))
trA = testAccuracy(Xtr,Ytr,theta)
teA = testAccuracy(Xte,Yte,theta)
print('\nTrain accuracy: ',trA,'\nTest accuracy: ',teA)
PlotBoundary(data,theta,d,[0,1])

STEP 4: Applying regularization

In [ ]:
# Play around with the lambda parameter with a high value of d
d = 1
l = 0
theta0,Xtr,Ytr = buildmodel(data.Xtrain,data.Ytrain,d,[0,1])
_,Xte,Yte = buildmodel(data.Xtest,data.Ytest,d,[0,1])
sol = minimize(costFunction,theta0,args=(Xtr,Ytr,l))
theta = np.reshape(sol.x,(Xtr.shape[1],Ytr.shape[1]))
trA = testAccuracy(Xtr,Ytr,theta)
teA = testAccuracy(Xte,Yte,theta)
print('\nTrain accuracy: ',trA,'\nTest accuracy: ',teA)
PlotBoundary(data,theta,d,[0,1])

**GRADIENT DESCENT**

STEP 5: Implementing gradient descent

In [ ]:
# We create a class gradient descent to use our own optimizer
# We have to define two main things, 1)the step 2)create the loop in the train method
class gradientDescent:
  
  def __init__(self,lr,maxiters,plot):
    self.lr = lr
    self.maxIters = maxiters
    self.plot = plot
    self.optHIST = []

  def train(self,F,f,X,Y,theta):
    # Loop
    
    self.optHIST = np.array(self.optHIST)
    self.plotEnd()
    return theta

  def step(self,theta,grad):
    #Step
    
    return theta
      
  def plotEnd(self):
    # A function that plots at the end the evolution of the function value and the step size
    if (self.plot == True):
      plt.show()
      plt.figure(1,figsize=(10,5))
      s = np.arange(0,self.maxIters,20)
      plt.subplot(1,2,1)
      plt.title('Function value vs iter')
      plt.scatter(s,self.optHIST[s,0])
      plt.subplot(1,2,2)
      plt.title('Step size vs iter')
      plt.scatter(s,self.optHIST[s,1])
      plt.show()

In [ ]:
# We try our minimizer
# Create the matrices and the gradient descent object
d = 2
l = 0.0
theta0,Xtr,Ytr = buildmodel(data.Xtrain,data.Ytrain,d,[0,1])
gd = gradientDescent(lr=0.01,maxiters=500,plot=True)

# Create the cost and gradient as handle functions to give them to our minimizer
F = lambda theta,X,Y: costFunction(theta,X,Y,l)
f = lambda theta,X,Y: gradient(theta,X,Y,l)

sol = gd.train(F,f,Xtr,Ytr,theta0)
theta = np.reshape(sol,[Xtr.shape[1],Ytr.shape[1]])
PlotBoundary(data,theta,d,[0,1])

_,Xte,Yte = buildmodel(data.Xtest,data.Ytest,d,[0,1])
tr        = testAccuracy(Xtr,Ytr,theta)
te        = testAccuracy(Xte,Yte,theta)
print('Train accuracy: ',tr,'\nTest accuracy: ',te)

STEP 7: Implementing stochastic gradient descent

In [ ]:
# We can create a class stochastic gradient descent which inherits from gradient descent
# We overwrite the loop (notice the only change is another loop inside iterating through the bath)
# We create a method create minibatch wich will create the batches at each iteration
class SGD(gradientDescent):
  def __init__(self,lr,maxepochs,batch,plot):
    gradientDescent.__init__(self,lr,maxepochs,plot)
    self.batchSz = batch

  def train(self,F,f,X,Y,theta,outFCN):
    # Here occurs the main loop of the minimization
    epoch = 0
    iter  = 0
    nB    = X.shape[0]//self.batchSz
    while epoch < self.maxIters:
      if (nB == 1 or nB == 0):
        order = np.arange(X.shape[0])
        nB = 1;
      else:
        order = np.random.permutation(X.shape[0])
      for i in range(nB):
        Xb,Yb = self.createMinibatch(X,Y,order,i)
        J     = F(theta,Xb,Yb)
        grad  = f(theta,Xb,Yb)
        gnorm = np.linalg.norm(grad)
        theta  = self.step(theta,grad)
        iter = iter + 1
      self.optHIST.append([J,gnorm*self.lr])
      epoch  += 1
    self.optHIST = np.array(self.optHIST)
    self.plotEnd()
    return theta

  def createMinibatch(self,X,Y,order,i):
    # Function that creates the minibatch
    cont = 0
    bs = self.batchSz
    if (i == X.shape[0]//bs-1):
      plus = X.shape[0]%bs
    else:
      plus = 0
    Xb = X[order[i*bs:(i+1)*bs+plus],:]
    Yb = Y[order[i*bs:(i+1)*bs+plus],:]
    return Xb,Yb

In [ ]:
# We test our new minimizer
d = 2
l = 0.0
idx = [0,1]
m = data.Xtrain.shape[0]
theta0,Xtr,Ytr = buildmodel(data.Xtrain,data.Ytrain,d,idx)
F = lambda theta,X,Y: costFunction(theta,X,Y,l)
f = lambda theta,X,Y: gradient(theta,X,Y,l)
sgd = SGD(lr=0.1,maxepochs=200,batch=m,plot=True)
sol = sgd.train(F,f,Xtr,Ytr,theta0,outFCN)
theta = np.reshape(sol,[Xtr.shape[1],Ytr.shape[1]])

PlotBoundary(data,theta,d,idx)
_,Xte,Yte = buildmodel(data.Xtest,data.Ytest,d,idx)
tr        = testAccuracy(Xtr,Ytr,theta)
te        = testAccuracy(Xte,Yte,theta)
print('Train accuracy: ',tr,'\nTest accuracy: ',te)